# Gaussian Process Regression

Assume data is multivariate normal 

$$\begin{bmatrix}
y \\
f_*
\end{bmatrix}
\sim \mathcal{N} \left(0, \begin{bmatrix}
K(X,X) + \sigma^2_o I & K(X,X_*) \\
K(X_*,X) & K(X_*,X_*)
\end{bmatrix}\right)$$

Then 

$$f_* | X, y, X_* =
K(X_*,X) (K(X,X) + \sigma^2_o I)^{-1} y$$



$$cov(f_*) = K(X_*,X_*) - K(X_*,X) \left(K(X,X) + \sigma^2_o I\right)^{-1} K(X,X_*)$$

Can we, instead of specifying stastics for "full fields" like $p(t,x)$, specify statistics for the anomaly $\delta p(t,x) = p(t,x) -\hat{p}(t,x)$. Which is by definition will be zero mean, i.e. $\hat{p}$ is a Reynolds average. 

In [1]:
using Plots, Interact, Dates, DataFrames, LinearAlgebra, IterativeSolvers
using Netatmo

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-8739429525992957460\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6639303263282300461\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/0f63de98390cd72174381640f96d318de90e7b72-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-8739429525992957460\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

CSV_ARCHIVE=/lustre/storeB/users/roels/netatmo
JSON_ARCHIVE=/lustre/storeB/project/metproduction/products/netatmo/


In [2]:
dtg       = DateTime(2019,5,1,0)
period    = Hour(72)
timerange = dtg:Minute(10):dtg + period
latrange  = 59.9:0.01:60  
lonrange  = 10.7:0.01:10.8
latrange  = 52:0.01:65  
lonrange  = 7:0.01:15
df = Netatmo.read(timerange, latrange=latrange, lonrange=lonrange)


Reading: CSV files100%|█████████████████████████████████| Time: 0:00:39


,id,time_utc
,String,Int64
1,enc:16:RjcKJHB7i6/T7H5OVIuVH5Rvz/aHlMD/avLgo96otmA13e3gpK8Y4zhUby5ZBnoG,1556668801
2,enc:16:3DlBWufvVn5Y8frQVAfukWMKjiAlqvWkrXFhDh2h4/gPusjU1K0wZQ8SF0Vejnr+,1556668804
3,enc:16:8USjqWB8mr5di7cAMak+ZVA4I5ogvXj4Hyjl0re2lkAa211EN/omZxnkBFiCLGgW,1556668804
4,enc:16:hKBCs8IdkHb8mk03bAN6S1pBpH/EwMySvpwiCwa6iSojUCaCikjagvjA7V075gmv,1556668802
5,enc:16:e3UN4VZxKA/A1KMFuogZD+ObBZncqIbKW9akOdDr/ijgbfR/63eODmI4vulbTEMD,1556668800
6,enc:16:kVOs7FeKYJ1aX6expeNmcXP/w25jy6JPsVcFoj05hFEAkZt0s6bj7MmBGPwkhXMF,1556668804
7,enc:16:240orv4vbl7hybq5D2T4SiENhVx0skjH9X/OWvN5u9YRSdVuPw8w2spiM+MAlLnn,1556668804
8,enc:16:vCwDIMkxacAeW1vhD3Vp9Mhi3x/efCxdv2kVlKdAb7jGOvBPl1dZU5rRByiDTE+s,1556668802
9,enc:16:hGa54LSm6bPloqs5SEZ/WTfVPKCeUAp0hQ2ZFy4O5ITRbZWxTqluMmY3R1Ne/Ua0,1556669276


# Temporal Gaussian process regression  

We use Gaussian process regression to compute the pressure anomaly as the difference between a smoothed surface pressure signal. 
The Squared exponential kernel is 

$$K_{se}(t_1,t_2) = \exp \left( - \frac{ |t_1-t_2|^2 }{2 l^2} \right) $$

The Ornstein Uhlenbeck kernel is 

$$K_{ou}(t_1,t_2) = \exp \left( - \frac{|t_1-t_2| }{l} \right) $$


## Impact of length scales

Try Lengthscale=24  in the $K_{se}$ kernel and see the  impact that the asssumed $\sigma_o$ has on the anomaly at the the end of the time window. I.e. $\sigma_o$ is more than just a regularization parameter. 
Check station 371 on 20180510

In [7]:
groupbyid = groupby(df,:id);
ranges = 1:1:48 
sigmaos = 0.0001:0.0001:0.001
indices=1:length(groupbyid)



mp = @manipulate for σₒ in sigmaos,  r in slider(ranges; label="Range"), index in indices 
    s = groupbyid[index]
    Kou(t₁,t₂) = exp(-1/2*abs(t₁-t₂)/(1000000*60*60*r))   # Ornstein–Uhlenbeck
    Kse(t₁,t₂) = exp(-1/2*(t₁-t₂)^2/(60*60*r)^2)          # squared-exponential 
    K  = [Kou(t₁,t₂) for t₁ in s[:time_utc], t₂ in s[:time_utc] ] 
    # Ks = [rbf(t1,t2) for t1 in datetime2unix.(timerange),     t2 in s1[:time_utc] ]     
    KpI = copy(K)  
    KpI[diagind(KpI)] .= diag(KpI) .+ (σₒ)^2
    timecg  = @elapsed q, cglog = cg(KpI,s[:pressure],log=true)       
    timenor = @elapsed q2 = KpI\s[:pressure]
    
    pshat2 = K* q2
    pshat = K*q
    
    #print(KpI-K)    
    scatter(unix2datetime.(s[:time_utc]),s[:pressure],marker=:o,label="Ps")     
    plot!(unix2datetime.(s[:time_utc]),pshat,label="cg")  
    plot!(unix2datetime.(s[:time_utc]),pshat2,label="nor")  
    
    plot!(title = "CG $cglog speedupfactor=$(round(timenor/timecg,digits=2))")
    plot!(legend=:bottomleft) 
    # end
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-f7d5eacd-d9fc-4c10-887d-7bf8f0cfb6b5", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["σₒ"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row-left"), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>10,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-center"), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:className=>"interact-flex-row-right"), 1)], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget"), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
5, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/home/roels/.julia/packages/Knockout/JE2Yq/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/roels/.julia/packages/Knockout/JE2Yq/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/roels/.julia/packages/InteractBase/l8cEC/src/../assets/all.js"), Asset("css", nothing, "/home/roels/.julia/packages/InteractBase/l8cEC/src/../assets/style.css"), Asset("css", nothing, "/home/roels/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0001\",\"0.0002\",\"0.0003\",\"0.0004\",\"0.0005\",\"0.0006\",\"0.0007\",\"0.0008\",\"0.0009\",\"0.001\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"knockout-component-f7d5eacd-d9fc-4c10-887d-7bf8f0cfb6b5\",\"id\":\"ob_45\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"knockout-component-f7d5eacd-d9fc-4c10-887d-7bf8f0cfb6b5\",\"id\":\"ob_44\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this